In [1]:
cd '/home/dfried/projects/onecommon/aaai2020/experiments'

/home/dfried/projects/onecommon/aaai2020/experiments


In [2]:
import models
import data
import utils
from domain import get_domain

In [3]:
domain = get_domain('one_common')
unk_threshold = 10
model_ty = models.get_model_type('rnn_reference_model')

ctx_encoder_type = 'rel_attn_encoder'
model_fname = 'rel_tsel_ref_dial_model/default_0_ep-4.th'
# model is actually in "serialized_models/{model_fname}"

In [4]:
seed = 0
corpus = model_ty.corpus_ty(domain, 'data/onecommon',
                            train='train_reference_{}.txt'.format(seed),
                            valid='valid_reference_{}.txt'.format(seed),
                            test='test_reference_{}.txt'.format(seed),
                            freq_cutoff=unk_threshold, verbose=True)

dataset data/onecommon/train_reference_0.txt, total 544674, unks 4134, ratio 0.76%
dataset data/onecommon/valid_reference_0.txt, total 70004, unks 720, ratio 1.03%
dataset data/onecommon/test_reference_0.txt, total 67008, unks 576, ratio 0.86%


In [5]:
corpus

In [6]:
model = utils.load_model(model_fname, map_location='cpu')

In [7]:
batchsize=10
validset, validset_stats = corpus.valid_dataset(batchsize)

In [8]:
validset_stats

{'n': 110141, 'nonpadn': 70004}

In [9]:
for bsz in [1, 2, 5, 10]:
    dset, dset_stats = corpus.valid_dataset(bsz)
    print(bsz, len(dset), dset_stats)

1 1038 {'n': 70004, 'nonpadn': 70004}
2 525 {'n': 81884, 'nonpadn': 70004}
5 217 {'n': 97798, 'nonpadn': 70004}
10 116 {'n': 109484, 'nonpadn': 70004}


In [ ]:
# ctx: this player's dots
# inpt / tgt: dialogue word indices (tgt are shifted by 1)
# ref_inpt

In [104]:
for (input_vals, word_idxs, referent_idxs, output_idx, scenario_id, real_ids, agent, chat_id) in corpus.train:
    if 'C_d0de3d0144df405e998ae718539ff9a4' == chat_id:
        print('found it')
        break

found it


In [107]:
import torch

In [109]:
torch.LongTensor(referent_idxs).resize(5, 10)

/home/dfried/anaconda2/envs/onecommon/lib/python3.6/site-packages/torch/tensor.py:287: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


tensor([[ 3,  6, 15,  0,  1,  0,  0,  0,  0,  0],
        [ 8, 12, 15,  1,  0,  0,  0,  0,  0,  0],
        [14, 14, 15,  0,  1,  0,  0,  0,  0,  0],
        [45, 47, 64,  0,  0,  0,  1,  0,  0,  0],
        [61, 62, 64,  0,  1,  0,  0,  0,  0,  0]])

In [90]:
trainset, trainse_stats = corpus.train_dataset(1)

In [93]:
for ix in range(len(trainset)):
    ctx, inpt, tgt, ref_inpt, ref_tgt, sel_tgt, \
    scenario_ids, real_ids, agents, chat_ids, sel_idxs = trainset[ix]
    if 'C_d0de3d0144df405e998ae718539ff9a4' in chat_ids:
        print("found it")
        break

found it


In [94]:
chat_ids

['C_d0de3d0144df405e998ae718539ff9a4']

In [95]:
inpt.size()

torch.Size([82, 1])

In [96]:
ref_inpt.size()

torch.Size([1, 5, 3])

In [97]:
ref_inpt[0]

tensor([[ 3,  6, 15],
        [ 8, 12, 15],
        [14, 14, 15],
        [45, 47, 64],
        [61, 62, 64]])

In [98]:
chat_ids

['C_d0de3d0144df405e998ae718539ff9a4']

In [99]:
' '.join(corpus.word_dict.get_word(idx) for idx in inpt[:,0])

'YOU: i see a dark grey dot with a slightly smaller darker dot below it <eos> THEM: i think i have that too . above the larger one and to its left i have a much smaller medium gray dot <eos> YOU: yes i have the smaller grey to the upper left as well . which do we click on ? the middle ? <eos> THEM: lets click on the large one you mentioned first . <eos> YOU: ok good luck <selection>'

In [100]:
ref_tgt

tensor([[[0, 1, 0, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0]]])

In [113]:
ix = 0
for ref_ix in range(ref_inpt.size(1)):
    print(ref_inpt[ix,ref_ix])
    print(' '.join(corpus.word_dict.get_word(idx) for idx in  inpt[:,ix][ref_inpt[ix,ref_ix,0]:ref_inpt[ix,ref_ix,1]+1]))

tensor([ 3,  6, 15])
a dark grey dot
tensor([ 8, 12, 15])
a slightly smaller darker dot
tensor([14, 14, 15])
it
tensor([45, 47, 64])
the smaller grey
tensor([61, 62, 64])
the middle


In [55]:
ref_inpt

tensor([[[ 3,  4, 28],
         [16, 18, 28],
         [25, 27, 28],
         [42, 44, 45]],

        [[ 1,  2, 12],
         [51, 51, 60],
         [55, 59, 60],
         [91, 93, 94]],

        [[ 4,  7, 15],
         [11, 13, 15],
         [56, 59, 72],
         [64, 67, 72]],

        [[ 1,  2, 10],
         [ 4,  5, 10],
         [24, 26, 41],
         [38, 40, 41]],

        [[24, 28, 40],
         [30, 32, 40],
         [37, 38, 40],
         [63, 65, 75]],

        [[31, 35, 41],
         [65, 69, 74],
         [72, 72, 74],
         [89, 89, 90]],

        [[ 1,  4, 15],
         [ 6,  8, 15],
         [13, 13, 15],
         [38, 40, 41]],

        [[13, 15, 27],
         [18, 20, 27],
         [22, 25, 27],
         [44, 45, 46]],

        [[ 3,  9, 10],
         [31, 31, 43],
         [37, 39, 43],
         [67, 71, 72]],

        [[13, 13, 25],
         [15, 17, 25],
         [19, 19, 25],
         [58, 62, 64]]])

In [54]:
ref_tgt

tensor([[[0, 0, 0, 1, 0, 1, 0],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0]],

        [[0, 1, 1, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0]],

        [[1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0]],

        [[0, 0, 0, 1, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1],
         [0, 0, 0, 1, 0, 0, 1]],

        [[1, 0, 0, 0, 0, 1, 0],
         [0, 1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0]],

        [[0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 1, 0, 0]],

        [[1, 0, 1, 0, 0, 0, 1],
         [0, 0, 1, 0, 0, 0, 0],
         [1, 0, 0, 0, 0, 0